In [1]:
import telebot
import numpy as np
import pandas as pd
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

In [2]:
# 5009761094:AAFZb4Z3Gr4NmQoZjZM22JvDfz6FNeEdqf8

In [3]:
# создаем бота

token = '5009761094:AAFZb4Z3Gr4NmQoZjZM22JvDfz6FNeEdqf8'
bot = telebot.TeleBot(token)

In [4]:
# функция, которая будет выполняться при отправке команды /start

@bot.message_handler(commands=['start'])
def start(message):
    bot.reply_to(message, "Я на связи")

In [5]:
# функция, котоорая выпполняется при отправке текстового сообщения

@bot.message_handler(func = lambda message: True)
def echo_all(message):
    bot.reply_to(message, pipe.predict([message.text.lower()])[0])

In [6]:
# загружаем готовую базу диалогов (с правками)

good = pd.read_csv('good.tsv', sep='\t')

In [7]:
# преобразуем в числовые векторы и запоминаем частоту каждого слова
vectorizer = TfidfVectorizer()
vectorizer.fit(good.context_0)
matrix_big =vectorizer.transform(good.context_0) 

In [8]:
# сокращаем размерность 

svd = TruncatedSVD(n_components = 300)
svd.fit(matrix_big)
matrix_small = svd.transform(matrix_big)

In [9]:
# пишем функцию для создания вероятностного распределения

def softmax(x):
    proba = np.exp(-x)
    return proba/sum(proba)

In [10]:
# создаем класс для случайного выбора одного из соседей

class NeighborSampler(BaseEstimator):
    def __init__(self, k=5, temperature=1.0):
        self.k = k
        self.temperature = temperature
    def fit(self, X, y):
        self.tree_ = BallTree(X)
        self.y_ = np.array(y)
    def predict(self, X, random_state=None):
        distances, indices = self.tree_.query(X, return_distance = True, k = self.k)
        result = []
        for distance, index in zip(distances, indices):
            result.append(np.random.choice(index, p = softmax(distance*self.temperature)))
        return self.y_[result]

In [11]:
# соединяем векторизацию, сокращение размерности и поиск соседей

ns = NeighborSampler()
ns.fit(matrix_small, good.reply)
pipe = make_pipeline(vectorizer, svd, ns)

In [ ]:
# запускаем бота

bot.polling()  